In [1]:
# Path to source code
%cd ../../

c:\Users\ajaoo\Desktop\Projects\Multivate-forecasting


In [2]:
# Imports for handling data
import os
import shutil
import numpy as np
import pandas as pd
from pathlib import Path
from itertools import cycle
# from sklearn.model_selection import TimeSeriesSplit, train_test_split

# Imports for machine learning
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse
# from sklearn.linear_model import LinearRegression
# from scipy.stats import spearmanr

# Imports for visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Progress bar
from tqdm.autonotebook import tqdm
# Enable progress apply for pandas
tqdm.pandas()


# Local imports for data loaders and models
from src.utils import plotting_utils
from src.dl.dataloaders import TimeSeriesDataModule
from src.dl.multivariate_models import SingleStepRNNConfig, SingleStepRNNModel
from src.transforms.target_transformations import AutoStationaryTransformer


pl.seed_everything(42)
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
    
torch.set_float32_matmul_precision('medium')

# Set default plotly template
import plotly.io as pio
pio.templates.default = "plotly_white"

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

%load_ext tensorboard

Seed set to 42


In [3]:
def format_plot(
    fig, legends=None, xlabel="Time", ylabel="Value", title="", font_size=15
):
    if legends:
        names = cycle(legends)
        fig.for_each_trace(lambda t: t.update(name=next(names)))
    fig.update_layout(
        autosize=False,
        width=900,
        height=500,
        title_text=title,
        title={"x": 0.5, "xanchor": "center", "yanchor": "top"},
        titlefont={"size": 20},
        legend_title=None,
        legend=dict(
            font=dict(size=font_size),
            orientation="h",
            yanchor="bottom",
            y=0.98,
            xanchor="right",
            x=1,
        ),
        yaxis=dict(
            title_text=ylabel,
            titlefont=dict(size=font_size),
            tickfont=dict(size=font_size),
        ),
        xaxis=dict(
            title_text=xlabel,
            titlefont=dict(size=font_size),
            tickfont=dict(size=font_size),
        ),
    )
    return fig


def mase(actual, predicted, insample_actual):
    mae_insample = np.mean(np.abs(np.diff(insample_actual)))
    mae_outsample = np.mean(np.abs(actual - predicted))
    return mae_outsample / mae_insample


def forecast_bias(actual, predicted):
    return np.mean(predicted - actual)


def plot_forecast(
    pred_df, forecast_columns, forecast_display_names=None, save_path=None
):
    if forecast_display_names is None:
        forecast_display_names = forecast_columns
    else:
        assert len(forecast_columns) == len(forecast_display_names)

    mask = ~pred_df[forecast_columns[0]].isnull()
    colors = px.colors.qualitative.Set2  # Using a different color palette
    act_color = colors[0]
    colors = cycle(colors[1:])

    fig = go.Figure()

    # Actual data plot
    fig.add_trace(
        go.Scatter(
            x=pred_df[mask].index,
            y=pred_df[mask].covidOccupiedMVBeds,
            mode="lines",
            marker=dict(size=6, opacity=0.5),
            line=dict(color=act_color, width=2),
            name="Actual COVID-19 MVBeds trends",
        )
    )

    # Predicted data plot
    for col, display_col in zip(forecast_columns, forecast_display_names):
        fig.add_trace(
            go.Scatter(
                x=pred_df[mask].index,
                y=pred_df.loc[mask, col],
                mode="lines+markers",
                marker=dict(size=4),
                line=dict(color=next(colors), width=2),
                name=display_col,
            )
        )

    return fig


def highlight_abs_min(s, props=""):
    return np.where(s == np.nanmin(np.abs(s.values)), props, "")

In [4]:
# Load and Prepare Data
data_path = Path("data/processed/merged_nhs_covid_data.csv")
data = pd.read_csv(data_path).drop("Unnamed: 0", axis=1)
data['date'] = pd.to_datetime(data['date'])

In [5]:
# check the unique values in the areaName column
data['areaName'].unique()

array(['East of England', 'London', 'Midlands',
       'North East and Yorkshire', 'North West', 'South East',
       'South West'], dtype=object)

In [6]:
# Select a different area name
selected_area = "London" # "London", "South East", "North West", "East of England", "South West", "West Midlands", "East Midlands", "Yorkshire and The Humber", "North East"
data_filtered = data[data['areaName'] == selected_area]

# Data Processing
data_filtered['date'] = pd.to_datetime(data_filtered['date'])
data_filtered.sort_values(by=["date", "areaName"], inplace=True)
data_filtered.drop(["areaName", "areaCode", "cumAdmissions","cumulative_confirmed", "cumulative_deceased", "population", "latitude", "longitude", "epi_week"], axis=1, inplace=True)

In [7]:
def add_rolling_features(df, window_size, columns, agg_funcs=None):
    if agg_funcs is None:
        agg_funcs = ['mean']
    added_features = {}
    for column in columns:
        for func in agg_funcs:
            roll_col_name = f"{column}_rolling_{window_size}_{func}"
            df[roll_col_name] = df[column].rolling(window_size).agg(func)
            if column not in added_features:
                added_features[column] = []
            added_features[column].append(roll_col_name)
    # Drop rows with NaN values which are the result of rolling window
    df.dropna(inplace=True)
    return df, added_features

# Configuration
window_size = 7
columns_to_roll = ["hospitalCases", "newAdmissions", "new_confirmed", "new_deceased"]
agg_funcs = ['mean', 'std']

# # Apply rolling features for each column
# data_filtered, added_features = add_rolling_features(data_filtered, window_size, columns_to_roll, agg_funcs)

# # Print added features for each column
# for column, features in added_features.items():
#     print(f"{column}: {', '.join(features)}")

In [8]:
# Define a function to add time-lagged features to the dataset
def add_lags(data, lags, features):
    added_features = []
    for feature in features:
        for lag in lags:
            new_feature = feature + f'_lag_{lag}'
            data[new_feature] = data[feature].shift(lag)
            added_features.append(new_feature)
    return data, added_features


lags = [1, 2, 3, 5, 7, 14, 21]

data_filtered, added_features = add_lags(data_filtered, lags, ['covidOccupiedMVBeds'])
data_filtered.dropna(inplace=True)

In [9]:
def create_temporal_features(df, date_column):
    df["month"] = df[date_column].dt.month
    df["day"] = df[date_column].dt.day
    df["day_of_week"] = df[date_column].dt.dayofweek
    return df


data_filtered = create_temporal_features(data_filtered, "date")
data_filtered.head()

,date,covidOccupiedMVBeds,hospitalCases,newAdmissions,new_confirmed,new_deceased,covidOccupiedMVBeds_lag_1,covidOccupiedMVBeds_lag_2,covidOccupiedMVBeds_lag_3,covidOccupiedMVBeds_lag_5,covidOccupiedMVBeds_lag_7,covidOccupiedMVBeds_lag_14,covidOccupiedMVBeds_lag_21,month,day,day_of_week
1767,2020-04-22,927.0,3768.0,225,399.0,110.0,972.0,1019.0,991.0,982.0,1023.0,941.0,0.0,4,22,2
1766,2020-04-23,897.0,3510.0,266,520.0,80.0,927.0,972.0,1019.0,992.0,1012.0,849.0,673.0,4,23,3
1765,2020-04-24,836.0,3281.0,171,375.0,75.0,897.0,927.0,972.0,991.0,982.0,1057.0,759.0,4,24,4
1764,2020-04-25,868.0,3198.0,137,332.0,57.0,836.0,897.0,927.0,1019.0,992.0,1035.0,824.0,4,25,5
1763,2020-04-26,831.0,3115.0,187,310.0,62.0,868.0,836.0,897.0,972.0,991.0,1035.0,800.0,4,26,6


In [10]:
data_filtered['date'] = pd.to_datetime(data_filtered['date'])
data_filtered = data_filtered.set_index('date')
data_filtered.head()

,covidOccupiedMVBeds,hospitalCases,newAdmissions,new_confirmed,new_deceased,covidOccupiedMVBeds_lag_1,covidOccupiedMVBeds_lag_2,covidOccupiedMVBeds_lag_3,covidOccupiedMVBeds_lag_5,covidOccupiedMVBeds_lag_7,covidOccupiedMVBeds_lag_14,covidOccupiedMVBeds_lag_21,month,day,day_of_week
date,,,,,,,,,,,,,,,
2020-04-22,927.0,3768.0,225,399.0,110.0,972.0,1019.0,991.0,982.0,1023.0,941.0,0.0,4,22,2
2020-04-23,897.0,3510.0,266,520.0,80.0,927.0,972.0,1019.0,992.0,1012.0,849.0,673.0,4,23,3
2020-04-24,836.0,3281.0,171,375.0,75.0,897.0,927.0,972.0,991.0,982.0,1057.0,759.0,4,24,4
2020-04-25,868.0,3198.0,137,332.0,57.0,836.0,897.0,927.0,1019.0,992.0,1035.0,824.0,4,25,5
2020-04-26,831.0,3115.0,187,310.0,62.0,868.0,836.0,897.0,972.0,991.0,1035.0,800.0,4,26,6


In [11]:
# Set the target variable
target = 'covidOccupiedMVBeds'

seasonal_period = 7
auto_stationary = AutoStationaryTransformer(seasonal_period=seasonal_period)

# Fit and transform the target column to make it stationary
data_stat = auto_stationary.fit_transform(data_filtered[[target]], freq="D")

# Replace the original target values with the transformed stationary values
data_filtered[target] = data_stat.values

# Print the transformed data to check
data_filtered.head()

,covidOccupiedMVBeds,hospitalCases,newAdmissions,new_confirmed,new_deceased,covidOccupiedMVBeds_lag_1,covidOccupiedMVBeds_lag_2,covidOccupiedMVBeds_lag_3,covidOccupiedMVBeds_lag_5,covidOccupiedMVBeds_lag_7,covidOccupiedMVBeds_lag_14,covidOccupiedMVBeds_lag_21,month,day,day_of_week
date,,,,,,,,,,,,,,,
2020-04-22,26.618059,3768.0,225,399.0,110.0,972.0,1019.0,991.0,982.0,1023.0,941.0,0.0,4,22,2
2020-04-23,26.279578,3510.0,266,520.0,80.0,927.0,972.0,1019.0,992.0,1012.0,849.0,673.0,4,23,3
2020-04-24,25.563608,3281.0,171,375.0,75.0,897.0,927.0,972.0,991.0,982.0,1057.0,759.0,4,24,4
2020-04-25,25.948043,3198.0,137,332.0,57.0,836.0,897.0,927.0,1019.0,992.0,1035.0,824.0,4,25,5
2020-04-26,25.509239,3115.0,187,310.0,62.0,868.0,836.0,897.0,972.0,991.0,1035.0,800.0,4,26,6


In [12]:
data_filtered.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 873 entries, 2020-04-22 to 2022-09-12
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   covidOccupiedMVBeds         873 non-null    float64
 1   hospitalCases               873 non-null    float64
 2   newAdmissions               873 non-null    int64  
 3   new_confirmed               873 non-null    float64
 4   new_deceased                873 non-null    float64
 5   covidOccupiedMVBeds_lag_1   873 non-null    float64
 6   covidOccupiedMVBeds_lag_2   873 non-null    float64
 7   covidOccupiedMVBeds_lag_3   873 non-null    float64
 8   covidOccupiedMVBeds_lag_5   873 non-null    float64
 9   covidOccupiedMVBeds_lag_7   873 non-null    float64
 10  covidOccupiedMVBeds_lag_14  873 non-null    float64
 11  covidOccupiedMVBeds_lag_21  873 non-null    float64
 12  month                       873 non-null    int32  
 13  day             

In [13]:
# Get the minimum and maximum date from the data
min_date = data_filtered.index.min()
max_date = data_filtered.index.max()
# Calculate the range of dates
date_range = max_date - min_date
print(f"Data ranges from {min_date} to {max_date} ({date_range.days} days)")

Data ranges from 2020-04-22 00:00:00 to 2022-09-12 00:00:00 (873 days)


In [14]:
# Filter data between the specified dates
start_date = "2020-05-01"
end_date = "2022-08-30"
data_filtered = data_filtered[start_date:end_date]

In [15]:
# selecting 1 year data for training and 2 months data for validation and 3 months data for testing
# Calculate the date ranges for train, val, and test sets
train_end = min_date + pd.Timedelta(days=date_range.days * 0.70)
val_end = train_end + pd.Timedelta(days=date_range.days * 0.15)

# Split the data into training, validation, and testing sets
train = data_filtered[data_filtered.index <= train_end]
val = data_filtered[(data_filtered.index > train_end) & (data_filtered.index < val_end)]
test = data_filtered[data_filtered.index > val_end]

# Calculate the percentage of dates in each dataset
total_sample = len(data_filtered)
train_sample = len(train) / total_sample * 100
val_sample = len(val) / total_sample * 100
test_sample = len(test) / total_sample * 100

print(f"Train: {train_sample:.2f}%, Validation: {val_sample:.2f}%, Test: {test_sample:.2f}%")
print(f"Train: {len(train)} samples, Validation: {len(val)} samples, Test: {len(test)} samples")
print(f"Max date in train: {train.index.max()}, Min date in validation: {val.index.min()}, Max date in test: {test.index.max()}")

Train: 70.77%, Validation: 15.38%, Test: 13.85%
Train: 603 samples, Validation: 131 samples, Test: 118 samples
Max date in train: 2021-12-24 00:00:00, Min date in validation: 2021-12-25 00:00:00, Max date in test: 2022-08-30 00:00:00


In [16]:
# plot the train data
fig = go.Figure()
fig.add_trace(go.Scatter(x=train.index, y=train['covidOccupiedMVBeds'], mode='lines', name='Train Data'))
fig.update_layout(title='Train Data for COVID-19 Occupied MV Beds in Midlands Region', xaxis_title='Date', yaxis_title='COVID-19 Occupied MV Beds', template='plotly_white')
fig.show()

In [17]:
# Concatenate the DataFrames
sample_df = pd.concat([train, val, test])

# Convert all the feature columns to float32
for col in sample_df.columns:
    sample_df[col] = sample_df[col].astype('float32')
    
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 852 entries, 2020-05-01 to 2022-08-30
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   covidOccupiedMVBeds         852 non-null    float32
 1   hospitalCases               852 non-null    float32
 2   newAdmissions               852 non-null    float32
 3   new_confirmed               852 non-null    float32
 4   new_deceased                852 non-null    float32
 5   covidOccupiedMVBeds_lag_1   852 non-null    float32
 6   covidOccupiedMVBeds_lag_2   852 non-null    float32
 7   covidOccupiedMVBeds_lag_3   852 non-null    float32
 8   covidOccupiedMVBeds_lag_5   852 non-null    float32
 9   covidOccupiedMVBeds_lag_7   852 non-null    float32
 10  covidOccupiedMVBeds_lag_14  852 non-null    float32
 11  covidOccupiedMVBeds_lag_21  852 non-null    float32
 12  month                       852 non-null    float32
 13  day             

In [18]:
columns_to_select = [
    "covidOccupiedMVBeds",
    # "hospitalCases",
    # "newAdmissions",
    # "new_confirmed",
    # "new_deceased",
    "covidOccupiedMVBeds_lag_1",
    "covidOccupiedMVBeds_lag_2",
    "covidOccupiedMVBeds_lag_3",
    "covidOccupiedMVBeds_lag_5",
    "covidOccupiedMVBeds_lag_7",
    "covidOccupiedMVBeds_lag_14",
    "covidOccupiedMVBeds_lag_21",
    "month",
    "day",
    "day_of_week",
]

In [19]:
sample_df = sample_df[columns_to_select]
sample_df.head()

,covidOccupiedMVBeds,covidOccupiedMVBeds_lag_1,covidOccupiedMVBeds_lag_2,covidOccupiedMVBeds_lag_3,covidOccupiedMVBeds_lag_5,covidOccupiedMVBeds_lag_7,covidOccupiedMVBeds_lag_14,covidOccupiedMVBeds_lag_21,month,day,day_of_week
date,,,,,,,,,,,
2020-05-01,24.006195,723.0,733.0,755.0,831.0,836.0,982.0,1057.0,5.0,1.0,4.0
2020-05-02,23.543913,712.0,723.0,733.0,800.0,868.0,992.0,1035.0,5.0,2.0,5.0
2020-05-03,22.978481,678.0,712.0,723.0,755.0,831.0,991.0,1035.0,5.0,3.0,6.0
2020-05-04,22.219740,638.0,678.0,712.0,733.0,800.0,1019.0,1046.0,5.0,4.0,0.0
2020-05-05,21.880533,587.0,638.0,678.0,723.0,755.0,972.0,1026.0,5.0,5.0,1.0


In [20]:
cols = list(sample_df.columns)
cols.remove("covidOccupiedMVBeds")
sample_df = sample_df[cols + ["covidOccupiedMVBeds"]]

In [21]:
target = "covidOccupiedMVBeds"
pred_df = pd.concat([train[[target]], val[[target]]])

In [22]:
datamodule = TimeSeriesDataModule(
    data=sample_df,
    n_val=val.shape[0],
    n_test=test.shape[0],
    window=7,  # 7 days window
    horizon=1,  # single step
    normalize="global",  # normalizing the data
    batch_size=32,
    num_workers=0,
)
datamodule.setup()

# Check a few batches from the training dataloader
train_loader = datamodule.train_dataloader()
for x, y in train_loader:
    print("Input batch shape:", x.shape)
    print("Output batch shape:", y.shape)
    break

Input batch shape: torch.Size([32, 7, 11])
Output batch shape: torch.Size([32, 1, 1])


In [23]:
# Setting up TensorBoard logger
logger = TensorBoardLogger("/notebook/multivariate/tb_logs", name="my_rnn_experiment")

%tensorboard --logdir tb_logs


rnn_config = SingleStepRNNConfig(
    rnn_type="GRU",
    input_size=11,  # 25 for multivariate time series
    hidden_size=32,  # hidden size of the RNN
    num_layers=10, # number of layers
    bidirectional=False, # bidirectional RNN
    learning_rate=1e-3,
)
model = SingleStepRNNModel(rnn_config)
model.float()

trainer = pl.Trainer(
    logger=logger,
    min_epochs=5,
    max_epochs=100,
    accelerator = "gpu",
    devices = 1,
    callbacks=[EarlyStopping(monitor="valid_loss", patience=10)],
)
trainer.fit(model, datamodule)

Reusing TensorBoard on port 6006 (pid 9184), started 1:02:22 ago. (Use '!kill 9184' to kill it.)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type    | Params
---------------------------------
0 | rnn  | GRU     | 61.3 K
1 | fc   | Linear  | 33    
2 | loss | MSELoss | 0     
---------------------------------
61.4 K    Trainable params
0         Non-trainable params
61.4 K    Total params
0.246     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [24]:
# Removing artifacts created during training
shutil.rmtree("/notebook/multivariate/tb_logs")

In [25]:
metric_record = []

In [26]:
predictions = trainer.predict(model, datamodule.test_dataloader())
predictions = torch.cat(predictions).squeeze().detach().numpy()
# De-normalizing the predictions
predictions = predictions * datamodule.train.std + datamodule.train.mean

actuals = test["covidOccupiedMVBeds"].values

assert (
    actuals.shape == predictions.shape
), "Mismatch in shapes between actuals and predictions"

algorithm_name = rnn_config.rnn_type

metrics = {
    "Algorithm": algorithm_name,
    "MAE": mae(actuals, predictions),
    "MSE": mse(actuals, predictions),
    "MASE": mase(actuals, predictions, train["covidOccupiedMVBeds"].values),
    "Forecast Bias": forecast_bias(actuals, predictions),
}

value_formats = ["{}", "{:.4f}", "{:.4f}", "{:.4f}", "{:.2f}"]
metrics = {
    key: format_.format(value)
    for key, value, format_ in zip(metrics.keys(), metrics.values(), value_formats)
}

pred_df_ = pd.DataFrame({f"Vanilla {algorithm_name}": predictions}, index=test.index)
pred_df = test.join(pred_df_)

metric_record.append(metrics)
print(metrics)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

{'Algorithm': 'GRU', 'MAE': '4.9943', 'MSE': '25.8500', 'MASE': '22.3799', 'Forecast Bias': '-4.99'}


In [27]:
fig = plot_forecast(
    pred_df,
    forecast_columns = [f"Vanilla {algorithm_name}"],
    forecast_display_names=[f"Vanilla {algorithm_name}"],
)
title = f"Forecasting COVID-19 MVBeds with {algorithm_name}"
fig = format_plot(fig, title=title)
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list(
            [
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all"),
            ]
        )
    ),

)
# save_path = f"images/forecast_multivarate_{algorithm_name}.png"
# pio.write_image(fig, save_path)
fig.show()

In [28]:
# plot the forecast of the training data and test data
fig = go.Figure()   
fig.add_trace(go.Scatter(x=train.index, y=train['covidOccupiedMVBeds'], mode='lines', name='Train Data'))   
fig.add_trace(go.Scatter(x=val.index, y=val['covidOccupiedMVBeds'], mode='lines', name='Validation Data'))
fig.add_trace(go.Scatter(x=test.index, y=test['covidOccupiedMVBeds'], mode='lines', name='Test Data'))
fig.add_trace(go.Scatter(x=pred_df.index, y=pred_df[f"Vanilla {algorithm_name}"], mode='lines', name=f"Vanilla {algorithm_name} Forecast")) 
fig.update_layout(title='Forecasting COVID-19 Occupied MV Beds in Midlands Region', xaxis_title='Date', yaxis_title='COVID-19 Occupied MV Beds', template='plotly_white')
fig.show()

In [29]:
fig = plot_forecast(
    pred_df,
    forecast_columns = [f"Vanilla {algorithm_name}"],
    forecast_display_names=[f"Vanilla {algorithm_name}"],
)
title = f"Forecasting COVID-19 MVBeds with {algorithm_name}"
fig = format_plot(fig, title=title)
fig.update_xaxes(
    type="date", range=["2022-05-01", "2022-07-31"], dtick="M1", tickformat="%b %Y"
)
save_path = f"images/forecast_multivarate_{algorithm_name}.png"
# pio.write_image(fig, save_path)
fig.show()

## Simulated annealing hyperparameter tunning

In [30]:
# Define the bounds for parameters
param_bounds = {
    "rnn_type": ["RNN","GRU","LSTM"],
    # hidden size between 32 and 128
    "hidden_size": (32, 128),
    "num_layers": (5, 30),
    "bidirectional": [True, False],
    # "learning_rate": (1e-5, 1e-1),
}

# Initial hyperparameters and temperature
initial_params = ["RNN", 10, 5, True]
initial_temp = 10

In [31]:
def objective(params):
    rnn_type, hidden_size, num_layers, bidirectional = params
    input_size = 11  # 25 for multivariate time series


    rnn_config = SingleStepRNNConfig(
        rnn_type=rnn_type,
        input_size=input_size,
        hidden_size=hidden_size,
        num_layers=num_layers,
        bidirectional=bidirectional,
        learning_rate=1e-3,
    )
    model = SingleStepRNNModel(rnn_config)

    # Training the model using PyTorch Lightning
    trainer = pl.Trainer(
        logger=logger,
        max_epochs=100,
        min_epochs=5,
        accelerator = "gpu",
        devices = 1,
        callbacks=[EarlyStopping(monitor="valid_loss", patience=10)],
    )
    trainer.fit(model, datamodule)
    
    %tensorboard --logdir tb_logs
    
    predictions = trainer.predict(model, datamodule.test_dataloader())
    predictions = torch.cat(predictions).squeeze().detach().numpy()
    # De-normalizing the predictions
    predictions = predictions * datamodule.train.std + datamodule.train.mean
    
    actuals = test["covidOccupiedMVBeds"].values
    
    algorithm_name = rnn_config.rnn_type
    
    metrics = {
        "Algorithm": algorithm_name,
        "MAE": mae(actuals, predictions),
        "MSE": mse(actuals, predictions),
        "MASE": mase(actuals, predictions, train["covidOccupiedMVBeds"].values),
        "Forecast Bias": forecast_bias(actuals, predictions),
    }
    
    return metrics["MAE"]


In [32]:
# Neighbor function to generate a new candidate solution
def neighbor(params):
    rnn_type, hidden_size, num_layers, bidirectional = params

    # Perturbations
    hidden_size = np.random.randint(*param_bounds["hidden_size"])
    num_layers = np.random.randint(*param_bounds["num_layers"])
    # learning_rate = np.random.uniform(*param_bounds["learning_rate"])
    rnn_type = np.random.choice(param_bounds["rnn_type"])
    bidirectional = bool(
        np.random.choice(param_bounds["bidirectional"])
    )  # Convert to native boolean
    
    # ensure that it returns the best parameters
    return [rnn_type, hidden_size, num_layers, bidirectional]

In [33]:
def simulated_annealing(initial_params, initial_temp, steps, cooling_factor=0.95):
    current_params = initial_params
    current_temp = initial_temp
    best_params = current_params
    best_score = objective(current_params)
    
    for step in range(steps):
        # Generate a new candidate solution
        candidate_params = neighbor(current_params)
        candidate_score = objective(candidate_params)
        
        # Calculate the acceptance probability
        acceptance_prob = np.exp((best_score - candidate_score) / current_temp)
        
        # Accept the candidate solution if it has a lower score
        if candidate_score < best_score:
            best_score = candidate_score
            best_params = candidate_params
        elif np.random.uniform() < acceptance_prob:
            best_score = candidate_score
            best_params = candidate_params
        
        # Update the current parameters
        current_params = candidate_params
        
        # Update the temperature with cooling schedule (geometric cooling for simulated annealing)
        current_temp *= cooling_factor
        
        print(f"Step {step+1}/{steps} - Best MAE: {best_score:.4f}")

    return best_params

In [35]:
shutil.rmtree("/notebook/multivariate/tb_logs")

In [36]:
# Run Simulated Annealing
best_params = simulated_annealing(initial_params, initial_temp, steps=250)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type    | Params
---------------------------------
0 | rnn  | RNN     | 3.0 K 
1 | fc   | Linear  | 21    
2 | loss | MSELoss | 0     
---------------------------------
3.0 K     Trainable params
0         Non-trainable params
3.0 K     Total params
0.012     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [ ]:
# Print the best parameters
print("Best parameters found:", best_params)

In [ ]:
# Training the model using the best hyperparameters
rnn_type = best_params[0][0]
hidden_size = best_params[0][1]
num_layers = best_params[0][2]
birdirectional = best_params[0][3]

input_size = 11 # 25 for multivariate time series

rnn_config = SingleStepRNNConfig(
    rnn_type=rnn_type,
    input_size=input_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    bidirectional=birdirectional,
    learning_rate=1e-3,
)
model = SingleStepRNNModel(rnn_config)

# Training the model using PyTorch Lightning
trainer = pl.Trainer(
    logger=logger,
    min_epochs=5,
    max_epochs=100,
    accelerator = "gpu",
    devices = 1,
    callbacks=[EarlyStopping(monitor="valid_loss", patience=10)],
)
trainer.fit(model, datamodule)

%tensorboard --logdir tb_logs

In [ ]:
shutil.rmtree("/notebook/multivariate/tb_logs")

In [ ]:
predictions = trainer.predict(model, datamodule.test_dataloader())
predictions = torch.cat(predictions).squeeze().detach().numpy()

# De-normalizing the predictions
predictions = predictions * datamodule.train.std + datamodule.train.mean

actuals = test["covidOccupiedMVBeds"].values

assert (
    actuals.shape == predictions.shape
), "Mismatch in shapes between actuals and predictions"

algorithm_name = rnn_config.rnn_type

metrics = {
    "Algorithm": f"{algorithm_name} (SA)",
    "MAE": mae(actuals, predictions),
    "MSE": mse(actuals, predictions),
    "MASE": mase(actuals, predictions, train["covidOccupiedMVBeds"].values),
    "Forecast Bias": forecast_bias(actuals, predictions),
}

pred_df_ = pd.DataFrame({f"{algorithm_name} (SA)": predictions}, index=test.index)
pred_df = test.join(pred_df_)


metric_record.append(metrics)

print(metrics)

In [ ]:
# Plotting the forecast
fig = plot_forecast(
    # simulated annealing model
    pred_df,
    forecast_columns=[f"{algorithm_name} (SA)"],
    forecast_display_names=[algorithm_name],
)

title = f"Forecasting multivarate COVID-19 MVBeds with {algorithm_name} (SA)"
fig = format_plot(fig, title=title)

fig.update_xaxes(
    type="date", range=["2021-10-01", "2021-12-31"], dtick="M1", tickformat="%b %Y"
)

save_path = f"images/forecast_multivarate_{algorithm_name}_sa.png"
# pio.write_image(fig, save_path)
fig.show()

In [ ]:
metric_df = pd.DataFrame(metric_record)
metric_df.info()

In [ ]:
metric_df[["MAE", "MSE", "MASE", "Forecast Bias"]] = metric_df[
    ["MAE", "MSE", "MASE", "Forecast Bias"]
].astype("float32")

In [ ]:
formatted = metric_df.style.format(
    {
        "MAE": "{:.4f}",
        "MSE": "{:.4f}",
        "MASE": "{:.4f}",
        "Forecast Bias": "{:.2f}%",
        "Time Elapsed": "{:.6f}",
    }
)
formatted = formatted.highlight_min(
    color="lightgreen", subset=["MAE", "MSE", "MASE"]
).apply(
    highlight_abs_min,
    props="color:black;background-color:lightgreen",
    axis=0,
    subset=["Forecast Bias"],
)
formatted